In [11]:
import pandas as pd
import re
from pathlib import Path

In [12]:
# Set base path
BASE_PATH = Path("D:/SEMESTER 6/PROJECT CBR/data")
RAW_PATH = BASE_PATH / "raw"
LOG_PATH = BASE_PATH / "logs"

# Buat folder jika belum ada
RAW_PATH.mkdir(parents=True, exist_ok=True)
LOG_PATH.mkdir(parents=True, exist_ok=True)

# Load data hasil scraping
df = pd.read_csv("D:/SEMESTER 6/PROJECT CBR/putusan_ma__2025-06-23.csv")

# Fungsi cleaning
def clean_text(text):
    text = str(text)
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # bisa disesuaikan
    return text.lower().strip()

# File log
log_file = open(LOG_PATH / "cleaning.log", "w", encoding="utf-8")

# Loop per dokumen
for i, row in df.iterrows():
    try:
        teks_asli = row.get('text_pdf', '')
        teks_bersih = clean_text(teks_asli)
        filename = RAW_PATH / f"case_{i+1:03}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(teks_bersih)

        if len(teks_bersih) / max(len(teks_asli), 1) >= 0.8:
            log_file.write(f"[OK] {filename.name}\n")
        else:
            log_file.write(f"[WARN] {filename.name} - <80% teks\n")
    except Exception as e:
        log_file.write(f"[ERROR] case_{i+1:03}.txt - {str(e)}\n")

log_file.close()
